# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1tGcX2qzS2GoaN5zFeD5DVJxqDmoQdmMs7QncZwFCEqU/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's totally optional whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_All code should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach._

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

import math

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db

import re

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
# any general notebook setup, like log formatting

In [9]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library
* [ ] Write at least one unit test that tests this distance calculation function. 
* Taxi data:
    * [ ] Use `requests`, BeautifulSoup (`bs4`), and `pandas` to programmatically download the required CSV files & load into memory.
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box]
* Using the function that calculates the distance between two coordinates in kilometers, add a column to each `pandas` DataFrame of data that calculates the distance between pickup and dropoff locations for each trip.
* Weather data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
        * You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 
* Take a look at the lecture notes from the `pandas` lecture for hints on helpful functionality

### Processing Uber Data

_Write some prose that tells the reader what you're about to do here._

In [12]:
def load_and_clean_uber_data(csv_file):
    
    df = pd.read_csv (csv_file)
    
    del df["Unnamed: 0"]
    del df["key"]
    df = df[df["pickup_longitude"] < -73.717047]
    df = df[df["pickup_longitude"] > -74.242330]
    df = df[df["dropoff_longitude"] < -73.717047]
    df = df[df["dropoff_longitude"] > -74.242330]
    df = df[df["pickup_latitude"] < 40.908524]
    df = df[df["pickup_latitude"] > 40.560445]
    df = df[df["dropoff_latitude"] < 40.908524]
    df = df[df["dropoff_latitude"] > 40.560445]
    
    # df.to_csv('uber_data.csv')
    return df
    
    # raise NotImplemented()
    
def get_uber_data():
    
    uber_dataframe = load_and_clean_uber_data(r'/Users/sandyfan/Desktop/project/uber_rides_sample.csv')
    # uber_dataframe = load_and_clean_uber_data(r'/content/gdrive/My Drive//uber_rides_sample.csv')
    
    # add_distance_column(uber_dataframe)
    
    return uber_dataframe

In [17]:
date = get_uber_data()["pickup_datetime"]

In [18]:
date = pd.to_datetime(date, format = "%Y-%m-%d %H:%M:%S UTC")

In [19]:
month_name_values = date.groupby([date.dt.year, date.dt.month]).count()

In [20]:
month_name_values[2014]

pickup_datetime
1     2451
2     2398
3     2697
4     2646
5     2600
6     2411
7     2285
8     2139
9     2340
10    2609
11    2351
12    2331
Name: pickup_datetime, dtype: int64

### Calculating distance
_Write some prose that tells the reader what you're about to do here._

In [ ]:
from math import radians, cos, sin, asin, sqrt, atan, atan2
# longitude and latitude of pickup and dropoff locations
def calculate_distance(Pickup_Longitude, Pickup_Latitude, Dropoff_Longitude, Dropoff_Latitude):
    # transfer numbers to radius
    Pickup_Longitude = radians(Pickup_Longitude)
    Pickup_Latitude = radians(Pickup_Latitude)
    Dropoff_Longitude = radians(Dropoff_Longitude)
    Dropoff_Latitude = radians(Dropoff_Latitude)
    # the formulas of calculate_distance
    dlon = Dropoff_Longitude - Pickup_Longitude
    dlat = Dropoff_Latitude - Pickup_Latitude
    a = sin(dlat/2)**2 + cos(Pickup_Latitude) * cos(Dropoff_Latitude) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    r = 6371 # The Average Radius of Earth
    Distance = c*r
    return Distance

In [4]:
def calculate_distance(from_coord, to_coord):
    
    
    
    raise NotImplemented()

In [6]:
def test_calculate_distance():
    
    
    
    raise NotImplemented()

In [ ]:
def add_distance_column(dataframe):
    
    
    
    raise NotImplemented()

### Processing Taxi Data

_Write some prose that tells the reader what you're about to do here._

In [40]:
def find_taxi_csv_urls():
    
    content = requests.get('https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page')

    soup = bs4.BeautifulSoup(content.text, 'html.parser')
    
    links = []
    for x in [a['href'] for a in soup.find_all('a')]:
        string = x
        pattern = re.compile(r"https://s3.amazonaws.com/nyc-tlc/trip\+\bdata/yell.*[\W_]csv")
        match = pattern.findall(string)
        if match != []:
            a = match[0]
            links.append(a)
            
    yellow_taxi = links[54:]   # get the links of all years that we need 
    
    
    return yellow_taxi
    
    # raise NotImplemented()


In [41]:
def get_month_taxi_data(url,year,month):
    
    yellow_link = pd.read_csv(url)
    #yellow_link = pd.read_csv(url,on_bad_lines='skip')
    
    #del yellow_link["Rate_Code"]
    #del yellow_link["store_and_forward"]
    #del yellow_link["mta_tax"]
    
    sample_data = yellow_link.sample(month_name_values[year][month])
    
    return sample_data
    
    
    # raise NotImplemented()

In [42]:
def get_and_clean_taxi_data_2015():      
    
    taxi_dataframes_2015 = []
    
    all_csv_urls = find_taxi_csv_urls()
    
    for i in range(6):    # set as 6 b/c there are only first six months uber data in 2015

        dataframe = get_month_taxi_data(all_csv_urls[i],2015,i+1)
        
        # add_distance_column(dataframe)
        
        taxi_dataframes_2015.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data_2015 = pd.concat(taxi_dataframes_2015)
    return taxi_data_2015

In [ ]:
yellow_taxi_2015 = get_and_clean_taxi_data_2015()
yellow_taxi_2015.to_csv('yellow_2015.csv')

In [ ]:
pd.read_csv('https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2014-01.csv')  #?? # only 2015 works

In [43]:
def get_and_clean_taxi_data_2014():      
    
    taxi_dataframes_2014 = []
    
    all_csv_urls = find_taxi_csv_urls()
    
    for i in range(12,24):

        dataframe = get_month_taxi_data(all_csv_urls[i],2014,i-11)
        
        # add_distance_column(dataframe)
        
        taxi_dataframes_2014.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data_2014 = pd.concat(taxi_dataframes_2014)
    return taxi_data_2014

In [ ]:
yellow_taxi_2014 = get_and_clean_taxi_data_2014()
yellow_taxi_2014.to_csv('yellow_2014.csv')

In [ ]:
def get_and_clean_taxi_data_2013():      
    
    taxi_dataframes_2013 = []
    
    all_csv_urls = find_taxi_csv_urls()
    
    for i in range(24,36):

        dataframe = get_month_taxi_data(all_csv_urls[i],2013,i-23)
        
        # add_distance_column(dataframe)
        
        taxi_dataframes_2013.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data_2013 = pd.concat(taxi_dataframes_2013)
    return taxi_data_2013

In [ ]:
yellow_taxi_2013 = get_and_clean_taxi_data_2013()
yellow_taxi_2013.to_csv('yellow_2013.csv')

In [17]:
def get_and_clean_taxi_data_2012():      
    
    taxi_dataframes_2012 = []
    
    all_csv_urls = find_taxi_csv_urls()
    
    for i in range(36,48):

        dataframe = get_month_taxi_data(all_csv_urls[i],2012,i-35)
        
        # add_distance_column(dataframe)
        
        taxi_dataframes_2012.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data_2012 = pd.concat(taxi_dataframes_2012)
    
    return taxi_data_2012

In [ ]:
yellow_taxi_2012 = get_and_clean_taxi_data_2012()
yellow_taxi_2012.to_csv('/content/gdrive/My Drive/yellow_2012.csv')

In [ ]:
def get_and_clean_taxi_data_2011():      
    
    taxi_dataframes_2011 = []
    
    all_csv_urls = find_taxi_csv_urls()
    
    for i in range(48,60):

        dataframe = get_month_taxi_data(all_csv_urls[i],2011,i-47)
        
        # add_distance_column(dataframe)
        
        taxi_dataframes_2011.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data_2011 = pd.concat(taxi_dataframes_2011)
    
    return taxi_data_2011

In [ ]:
yellow_taxi_2011 = get_and_clean_taxi_data_2011()
yellow_taxi_2011.to_csv('/content/gdrive/My Drive/yellow_2011.csv')

In [ ]:
def get_and_clean_taxi_data_2010():      
    
    taxi_dataframes_2010 = []
    
    all_csv_urls = find_taxi_csv_urls()
    
    for i in range(60,72):

        dataframe = get_month_taxi_data(all_csv_urls[i],2010,i-59)
        
        # add_distance_column(dataframe)
        
        taxi_dataframes_2010.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data_2010 = pd.concat(taxi_dataframes_2010)
    return taxi_data_2010

In [ ]:
yellow_taxi_2010 = get_and_clean_taxi_data_2010()
yellow_taxi_2010.to_csv('/content/gdrive/My Drive/yellow_2010.csv')

In [ ]:
def get_and_clean_taxi_data_2009():      
    
    taxi_dataframes_2009 = []
    
    all_csv_urls = find_taxi_csv_urls()
    
    for i in range(72,84):

        dataframe = get_month_taxi_data(all_csv_urls[i],2009,i-71)
        
        # add_distance_column(dataframe)
        
        taxi_dataframes_2009.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data_2009 = pd.concat(taxi_dataframes_2009)
    return taxi_data_2009

In [ ]:
yellow_taxi_2009 = get_and_clean_taxi_data_2009()
yellow_taxi_2009.to_csv('/content/gdrive/My Drive/yellow_2009.csv')

In [48]:
data_taxi_2009 = pd.read_csv (r'/Users/sandyfan/Desktop/project/yellow_2009.csv')
data_taxi_2010 = pd.read_csv (r'/Users/sandyfan/Desktop/project/yellow_2010.csv')
data_taxi_2011 = pd.read_csv (r'/Users/sandyfan/Desktop/project/yellow_2011.csv')
data_taxi_2012 = pd.read_csv (r'/Users/sandyfan/Desktop/project/yellow_2012.csv')
data_taxi_2013 = pd.read_csv (r'/Users/sandyfan/Desktop/project/yellow_2013.csv')
data_taxi_2014 = pd.read_csv (r'/Users/sandyfan/Desktop/project/yellow_2014.csv')
data_taxi_2015 = pd.read_csv (r'/Users/sandyfan/Desktop/project/yellow_2015.csv')

In [49]:
data_taxi_2009  

,Unnamed: 0,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt
0,4065342,DDS,2009-01-18 18:19:35,2009-01-18 18:42:53,2,7.30,-73.978317,40.788407,NaN,NaN,-73.982605,40.723203,CASH,19.7,0.0,NaN,0.0,0.0,19.7
1,493713,VTS,2009-01-17 16:12:00,2009-01-17 16:13:00,2,0.51,-73.977240,40.742695,NaN,NaN,-73.972213,40.745687,CASH,4.1,0.0,NaN,0.0,0.0,4.1
2,5550814,DDS,2009-01-06 21:22:34,2009-01-06 21:35:21,2,2.90,-73.973698,40.750343,NaN,NaN,-74.005725,40.737338,CASH,9.7,0.5,NaN,0.0,0.0,10.2
3,5192016,CMT,2009-01-29 21:56:26,2009-01-29 22:10:22,1,4.80,-73.948329,40.774314,NaN,NaN,-73.897864,40.816029,Cash,14.2,0.0,NaN,0.0,0.0,14.2
4,9714192,CMT,2009-01-18 22:59:40,2009-01-18 23:09:47,1,2.80,0.000000,0.000000,NaN,NaN,0.000000,0.000000,Cash,9.4,0.0,NaN,0.0,0.0,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30072,11259277,VTS,2009-12-23 20:18:00,2009-12-23 20:32:00,5,0.87,-73.988617,40.759198,NaN,NaN,-73.989190,40.753337,Credit,8.5,0.5,0.5,1.0,0.0,10.5
30073,11792871,VTS,2009-12-20 18:26:00,2009-12-20 18:38:00,3,1.94,-74.005055,40.741005,NaN,NaN,-74.010135,40.714487,Credit,8.5,0.0,0.5,3.0,0.0,12.0
30074,9077466,VTS,2009-12-23 23:06:00,2009-12-23 23:27:00,5,8.52,-73.862717,40.769072,NaN,NaN,-73.978287,40.766497,CASH,20.9,0.5,0.5,0.0,0.0,21.9
30075,7827648,CMT,2009-12-04 18:43:50,2009-12-04 19:02:20,2,2.70,-73.989711,40.725641,NaN,0.0,-73.972552,40.756052,Cash,11.7,1.0,0.5,0.0,0.0,13.2


In [24]:
data_taxi_2009 = data_taxi_2009.rename(columns={"vendor_name": "vendor_id", "Trip_Pickup_DateTime": "pickup_datetime",
                               "Trip_Dropoff_DateTime": "dropoff_datetime","Passenger_Count": "passenger_count",
                               "Trip_Distance": "trip_distance","Start_Lon": "pickup_longitude",
                               "Start_Lat": "pickup_latitude","Rate_Code": "rate_code",
                               "store_and_forward": "store_and_fwd_flag","End_Lon": "dropoff_longitude",
                               "End_Lat": "dropoff_latitude","Payment_Type": "payment_type",
                               "Fare_Amt": "fare_amount","Tip_Amt": "tip_amount","surcharge":"extra",
                               "Tolls_Amt": "tolls_amount","Total_Amt": "total_amount"})

In [15]:
data_taxi_2015.head()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,9978167,2,2015-01-05 15:45:31,2015-01-05 15:54:55,1,2.32,-73.983727,40.765659,1,N,-74.008148,40.748619,1,10.0,0.0,0.5,2.0,0.0,0.3,12.8
1,11889587,1,2015-01-22 21:22:13,2015-01-22 21:25:33,1,0.20,-74.005447,40.717461,1,N,-74.008385,40.719444,2,3.5,0.5,0.5,0.0,0.0,0.3,4.8
2,2483671,2,2015-01-01 01:00:24,2015-01-01 01:12:22,5,1.61,-73.990288,40.752232,1,N,-73.995926,40.736336,2,9.0,0.5,0.5,0.0,0.0,0.3,10.3
3,7172271,2,2015-01-08 07:02:36,2015-01-08 07:16:32,1,2.22,-73.962936,40.758141,1,N,-73.991280,40.750053,1,11.0,0.0,0.5,2.2,0.0,0.3,14.0
4,10000885,2,2015-01-08 09:07:44,2015-01-08 09:13:55,1,0.65,-73.977654,40.761971,1,N,-73.977013,40.757851,2,5.5,0.0,0.5,0.0,0.0,0.3,6.3


In [31]:
frames = [data_taxi_2009,data_taxi_2010,data_taxi_2011,data_taxi_2012,data_taxi_2013,data_taxi_2014,data_taxi_2015]
    
taxi_data_new = pd.concat(frames)

In [45]:
frames = [data_taxi_2009,data_taxi_2010,data_taxi_2011,data_taxi_2012,data_taxi_2013,data_taxi_2014]
taxi_0910 = pd.concat(frames)

In [47]:
data_taxi_2014

,Unnamed: 0,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,2964327,VTS,2014-01-26 19:22:00,2014-01-26 19:26:00,6,0.80,-73.981148,40.765157,1,NaN,-73.971685,40.765702,CSH,4.5,0.0,0.5,0.00,0.0,5.00
1,377021,CMT,2014-01-12 14:48:41,2014-01-12 15:04:54,1,1.80,-73.984608,40.768527,1,N,-74.000387,40.758440,CRD,11.5,0.0,0.5,1.00,0.0,13.00
2,2367154,VTS,2014-01-24 17:32:00,2014-01-24 17:37:00,1,0.61,-73.983455,40.772122,1,NaN,-73.989555,40.774825,CSH,5.5,1.0,0.5,0.00,0.0,7.00
3,9050355,VTS,2014-01-10 19:17:00,2014-01-10 19:26:00,2,1.39,-74.014420,40.715597,1,NaN,-73.998335,40.715822,CRD,8.0,1.0,0.5,2.25,0.0,11.75
4,12077792,CMT,2014-01-15 09:26:54,2014-01-15 09:29:27,2,0.40,-73.978912,40.766900,1,N,-73.986017,40.769986,CSH,4.0,0.0,0.5,0.00,0.0,4.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29253,12836372,VTS,2014-12-31 10:15:00,2014-12-31 10:18:00,1,0.66,-73.962042,40.767667,1,NaN,-73.964715,40.772897,CRD,4.5,0.0,0.5,1.12,0.0,6.12
29254,4487139,CMT,2014-12-19 08:56:15,2014-12-19 09:14:32,2,1.60,-73.957183,40.783043,1,N,-73.962669,40.767424,CRD,12.5,0.0,0.5,2.60,0.0,15.60
29255,1855284,VTS,2014-12-04 07:59:00,2014-12-04 08:12:00,5,1.45,-73.971790,40.757320,1,NaN,-73.985750,40.747695,CRD,9.5,0.0,0.5,1.90,0.0,11.90
29256,12871581,VTS,2014-12-30 20:22:00,2014-12-30 20:31:00,2,2.75,-74.006000,40.735572,1,NaN,-73.988775,40.765092,CSH,10.0,0.5,0.5,0.00,0.0,11.00


In [46]:
taxi_0910

,Unnamed: 0,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,...,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,4065342,DDS,2009-01-18 18:19:35,2009-01-18 18:42:53,2.0,7.30,-73.978317,40.788407,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,493713,VTS,2009-01-17 16:12:00,2009-01-17 16:13:00,2.0,0.51,-73.977240,40.742695,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5550814,DDS,2009-01-06 21:22:34,2009-01-06 21:35:21,2.0,2.90,-73.973698,40.750343,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5192016,CMT,2009-01-29 21:56:26,2009-01-29 22:10:22,1.0,4.80,-73.948329,40.774314,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9714192,CMT,2009-01-18 22:59:40,2009-01-18 23:09:47,1.0,2.80,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29253,12836372,VTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-73.964715,40.772897,CRD,4.5,0.0,0.5,1.12,0.0,6.12
29254,4487139,CMT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,N,-73.962669,40.767424,CRD,12.5,0.0,0.5,2.60,0.0,15.60
29255,1855284,VTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-73.985750,40.747695,CRD,9.5,0.0,0.5,1.90,0.0,11.90
29256,12871581,VTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-73.988775,40.765092,CSH,10.0,0.5,0.5,0.00,0.0,11.00


In [32]:
taxi_data_new

,Unnamed: 0,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,...,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,RateCodeID,improvement_surcharge
0,4065342,DDS,2009-01-18 18:19:35,2009-01-18 18:42:53,2.0,7.30,-73.978317,40.788407,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,493713,VTS,2009-01-17 16:12:00,2009-01-17 16:13:00,2.0,0.51,-73.977240,40.742695,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5550814,DDS,2009-01-06 21:22:34,2009-01-06 21:35:21,2.0,2.90,-73.973698,40.750343,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5192016,CMT,2009-01-29 21:56:26,2009-01-29 22:10:22,1.0,4.80,-73.948329,40.774314,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9714192,CMT,2009-01-18 22:59:40,2009-01-18 23:09:47,1.0,2.80,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13509,6119923,NaN,NaN,NaN,1.0,3.30,-73.986435,40.752533,NaN,N,...,NaN,NaN,NaN,NaN,NaN,1.0,2015-06-17 22:56:19,2015-06-17 23:08:15,1.0,0.3
13510,9901802,NaN,NaN,NaN,1.0,4.00,-73.979218,40.740196,NaN,N,...,NaN,NaN,NaN,NaN,NaN,1.0,2015-06-26 06:40:26,2015-06-26 06:51:55,1.0,0.3
13511,6802140,NaN,NaN,NaN,1.0,6.50,0.000000,0.000000,NaN,N,...,NaN,NaN,NaN,NaN,NaN,1.0,2015-06-20 14:07:43,2015-06-20 14:33:44,1.0,0.3
13512,7113434,NaN,NaN,NaN,1.0,0.30,-73.977600,40.746559,NaN,N,...,NaN,NaN,NaN,NaN,NaN,1.0,2015-06-13 13:47:13,2015-06-13 13:49:22,1.0,0.3


In [ ]:
def combine_and_clean_all_data():
    
    frames = [data_taxi_2009,data_taxi_2010,data_taxi_2011,data_taxi_2012,data_taxi_2013,data_taxi_2014,data_taxi_2015]
    
    taxi_data_uncleaned = pd.contact(all_taxi_dataframes)
    
    taxi_data = pd.DataFrame(taxi_data_uncleaned, columns = ["pickup_datetime","dropoff_datetime",
                                                            "passenger_count","trip_distance","pickup_longitude",
                                                            "pickup_latitude","dropoff_longitude","dropoff_latitude",
                                                            "fare_amount","extra"])
    
    
    
    return taxi_data
    
    

In [ ]:
def get_and_clean_taxi_data():        # 这格不跑
    
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    
    for csv_url in all_csv_url:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

### Processing Weather Data¶

_Write some prose that tells the reader what you're about to do here._

In [11]:
def clean_month_weather_data_hourly(csv_file):
    raise NotImplemented()
    
    
def clean_month_weather_data_daily(csv_file):
    raise NotImplemented()
    

def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["TODO"]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather, daily_weather = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [14]:
engine = db.create_engine(DATABASE_URL)

In [15]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [16]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_Write some prose that tells the reader what you're about to do here._

In [17]:
def dataframes_to_table():
    raise NotImplemented()

In [ ]:
table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

dataframes_to_table(table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [18]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.
* [ ] Come up with 3 questions on your own that can be answered based on the data in the 4 tables. Create at least one visualization to answer each question. At least one visualization should require data from at least 3 tables.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_You don't have to query the data directly from the database. You can just re-use the pandas DataFrame that you created in Part 1._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [19]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
plot_visual_n(some_dataframe)